In [ ]:
import os
import re
import time

PROJECTION = "knn_distance_2"
NUM_RUNS = 100
DATASET = "WORD_VECTOR"
SAMPLE_STEP = 50
MAX_SAMPLES = 500
WITH_REPLACEMENT = True
N_JOBS = 1
N_CUBES = 10
P_OVERLAP = 0.5
N_CLUSTERS = 3

In [ ]:
import json
import time
from collections import Counter
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import pandas as pd
import pytz
import numpy as np
import seaborn as sns
from sklearn.metrics import roc_auc_score
from scipy.stats import spearmanr
from datetime import date, datetime
import calendar
from scipy.optimize import minimize_scalar
from tqdm import tqdm
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from umap.umap_ import UMAP
import networkx as nx
import os
from sklearn.cluster import AgglomerativeClustering
from networkx.algorithms.approximation.treewidth import treewidth_min_fill_in
import kmapper as km
import os
from sklearn.cluster import AgglomerativeClustering
from networkx.algorithms.approximation.treewidth import treewidth_min_fill_in
from networkx.algorithms.centrality import estrada_index
from networkx.algorithms.cycles import cycle_basis 
import gensim.downloader as api
import tensorflow as tf

import logging
logging.getLogger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

sns.set(style='darkgrid')
pd.options.display.max_rows = 100
pd.options.display.max_columns = 20


def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

def get_mnist_dataset(skip=1):
    mnist_path = 'fashion'

    raw_Xtrain, raw_ytrain = load_mnist(mnist_path, kind='train')
    raw_Xtest, raw_ytest = load_mnist(mnist_path, kind='t10k')

    raw_Xtrain = raw_Xtrain / 255
    raw_Xtest = raw_Xtest / 255

    train_indices = np.random.permutation(range(raw_Xtrain.shape[0]))[::skip]
    test_indices = np.random.permutation(range(raw_Xtest.shape[0]))[::skip]

    raw_Xtrain = raw_Xtrain[train_indices, :]
    raw_ytrain = raw_ytrain[train_indices]

    raw_Xtest = raw_Xtest[test_indices, :]
    raw_ytest = raw_ytest[test_indices]
    return raw_Xtrain, raw_ytrain, raw_Xtest, raw_ytest

def get_word_vector_dataset():

    model = api.load("glove-wiki-gigaword-50")
    vocab = list(model.vocab.keys())
    embeddings = np.array([model.get_vector(w) for w in vocab])

    D = {"vocab": vocab, "embeddings": embeddings}
    return D["embeddings"], np.array(D["vocab"])



In [ ]:


def build_mapper_graph(data, labels, title, visualize, projection, verbose=0):
    path_html = "/Users/dshiebler/workspace/data/mapper/{}.html".format(title)

    # Initialize
    mapper = km.KeplerMapper(verbose=verbose)

    if projection == "pca_{}":
        n_components = int(projection.split("_")[-1])
        projection = PCA(n_components=n_components)
    elif projection == "umap_1":
        n_components = int(projection.split("_")[-1])
        projection = UMAP(n_components=n_components)
        
    projected_data = mapper.fit_transform(data, projection=projection)

    # Create dictionary called 'graph' with nodes, edges and meta-information
    cover = km.Cover(n_cubes=N_CUBES, perc_overlap=P_OVERLAP)
    graph = mapper.map(projected_data, data, cover=cover, clusterer=AgglomerativeClustering(N_CLUSTERS))

    # Visualize it
    if visualize:
        path_html = mapper.visualize(
          graph, path_html=path_html,
          title=title, custom_tooltips=np.array([str(l) for l in labels]))
        out = graph, path_html
    else:
        out = graph
    return out


def nodes_in_component(component, graph):
    return sum([len(graph["nodes"][cluster]) for cluster in component])

def connected_component_node_counts(graph, nx_graph):
    cc_clusters = [cc for cc in nx.connected_components(nx_graph)]
    return [nodes_in_component(cc, graph) for cc in cc_clusters]

def resample(X, n_samples, with_replacement):
    indices = np.arange(0, X.shape[0])
    return X[np.random.choice(indices, size=n_samples, replace=with_replacement)]


def get_metrics(n_samples):
    return {
        "num_cc_p0": lambda graph, nx_graph: len([cc for cc in connected_component_node_counts(graph, nx_graph)]),
        "density": lambda graph, nx_graph: nx.density(nx_graph),
        "estrada_index": lambda graph, nx_graph: estrada_index(nx_graph),
        "num_basis_cycles": lambda graph, nx_graph: len(cycle_basis(nx_graph))
    }


def get_metric_values(n_samples):
    if N_JOBS == 1:
        print("calling get_graph with n_samples={}".format(n_samples))

    metric_to_sample_values = {metric_name: {n_samples: []} for metric_name in get_metrics(1).keys()}

    iterator = tqdm(range(NUM_RUNS)) if N_JOBS == 1 else range(NUM_RUNS)
    for i in iterator:
        data = resample(X, n_samples, with_replacement=WITH_REPLACEMENT)
        graph = build_mapper_graph(
            data, labels=Y, projection=PROJECTION, title="my_html-{}".format(i), visualize=False, verbose=0)
        nx_graph = km.adapter.to_nx(graph)
        for metric_name, metric_fn in get_metrics(n_samples).items():
            metric_to_sample_values[metric_name][n_samples].append(metric_fn(graph, nx_graph))

    return metric_to_sample_values



In [ ]:
if "MNIST_UMAP_" in DATASET:
    n_components = int(DATASET.split("_")[-1])
    rawX, Y, _, _ = get_mnist_dataset(skip=1)
    X = UMAP(n_components=n_components, n_neighbors=15).fit_transform(rawX, y=Y)
elif DATASET == "WORD_VECTOR":
    X, Y = get_word_vector_dataset()
else:
    raise ValueError("Dataset not recognized")

print("{} X.shape: {} Y.shape: {}".format(DATASET, X.shape, Y.shape))


In [ ]:
from parallel_process import parallel_process


print("Computing metric_to_sample_values_list with N_JOBS={}...".format(N_JOBS))
if N_JOBS == 1:
    metric_to_sample_values_list = []
    for n_samples in tqdm(range(SAMPLE_STEP, MAX_SAMPLES, SAMPLE_STEP)):
        metric_to_sample_values_list.append(get_metric_values(n_samples))
else:
    metric_to_sample_values_list = parallel_process(
        array=range(SAMPLE_STEP, MAX_SAMPLES, SAMPLE_STEP),
        function=get_metric_values,
        n_jobs=N_JOBS)
print("metric_to_sample_values_list computed!")


out = {}
out["metric_to_sample_values"] = {metric_name: {} for metric_name in get_metrics(1).keys()}
for metric_to_sample_values in metric_to_sample_values_list:
    for metric in out["metric_to_sample_values"]:
        out["metric_to_sample_values"][metric].update(metric_to_sample_values[metric])


In [ ]:
out = {}
out["metric_to_sample_values"] = {metric_name: {} for metric_name in get_metrics(1).keys()}
for metric_to_sample_values in metric_to_sample_values_list:
    for metric in out["metric_to_sample_values"]:
        out["metric_to_sample_values"][metric].update(metric_to_sample_values[metric])
